# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = df_train.append(df_test)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

In [5]:
pd.DataFrame({'meam':df[int_features].mean(),
             'Max':df[int_features].max(),
              'nunique':df[int_features].nunique()})

,meam,Max,nunique
Pclass,2.294882,3,3
SibSp,0.498854,8,7
Parch,0.385027,9,8


In [6]:
pd.DataFrame({'meam':df[float_features].mean(),
             'Max':df[float_features].max(),
              'nunique':df[float_features].nunique()})

,meam,Max,nunique
Age,29.881138,80.0000,98
Fare,33.295479,512.3292,281


In [7]:
pd.DataFrame({'meam':df[object_features].mean(),
             'Max':df[object_features].max(),
              'nunique':df[object_features].nunique()})

,meam,Max,nunique
Cabin,NaN,NaN,186
Embarked,NaN,NaN,3
Name,NaN,"van Melkebeke, Mr. Philemon",1307
Sex,NaN,male,2
Ticket,NaN,WE/P 5735,929


### 答
在做這些統計運算時，在數值上基本上沒甚麼問題。  
但類別上卻不存在**平均數mean**，e.g.【蘋果、貓、樹】的平均數是甚麼？  
另一方面值得注意的是：類別雖然適用Max運算，但其計算方式是對字元的ASCII大小來做比較，對資料的了解幫助不大。(更傾向於計算眾數與其數量)

----

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

### 答
* 座標類型資料。  

像是絕對位置的經緯度資料，`性質`上跟**類別型**的地區資料相近，但`表示方式`卻是**數值型**的型態。  
往往能帶出更細緻的資料關係但處理上必細心應對。  

三個類別中我認為最複雜的是**object型特徵**：比起直接了解數值資料代表的意義，類別資料必須考量各個值彼此關係，這關係到數值化時的順序選擇；再者像是使用One_hot_encoding時，也要考慮資料量夠不夠支撐模型訓練以免發生維度詛咒之類的狀況。  
雖然這樣說，但數值的資料倒也不容易，畢竟還要考慮其分布情形，做各種轉換以維持最終模型的健康。